## 1. 前向传播



$h_t = A_th_{t-1} + B_tx_t$

$y_t = C_th_t$

$y_t = C_t(A_th_{t-1} + B_tx_t) = C_t \times h_{old} + C_tB_tx_t$

假设输入为 $X = [x_0, x_1, x_2, x_3, x_4, x_5, x_6, x_7]$， 输出为$Y = [y_0, y_1, y_2, y_3, y_4, y_5, y_6, y_7]$

则，$Y = MX$

$M = \left[
\begin{array}{lll|lll|lll}
  C_0^T A_{0:0} B_0 & & & & & & & & \\
  C_1^T A_{1:0} B_0 & C_1^T A_{1:1} B_1 & & & & & & & \\
  C_2^T A_{2:0} B_0 & C_2^T A_{2:1} B_1 & C_2^T A_{2:2} B_2 & & & & & & \\
  \hline
  C_3^T A_{3:0} B_0 & C_3^T A_{3:1} B_1 & C_3^T A_{3:2} B_2 & C_3^T A_{3:3} B_3 & & & & & \\
  C_4^T A_{4:0} B_0 & C_4^T A_{4:1} B_1 & C_4^T A_{4:2} B_2 & C_4^T A_{4:3} B_3 & C_4^T A_{4:4} B_4 & & & & \\
  C_5^T A_{5:0} B_0 & C_5^T A_{5:1} B_1 & C_5^T A_{5:2} B_2 & C_5^T A_{5:3} B_3 & C_5^T A_{5:4} B_4 & C_5^T A_{5:5} B_5 & & & \\
  \hline
  C_6^T A_{6:0} B_0 & C_6^T A_{6:1} B_1 & C_6^T A_{6:2} B_2 & C_6^T A_{6:3} B_3 & C_6^T A_{6:4} B_4 & C_6^T A_{6:5} B_5 & C_6^T A_{6:6} B_6 & & \\
  C_7^T A_{7:0} B_0 & C_7^T A_{7:1} B_1 & C_7^T A_{7:2} B_2 & C_7^T A_{7:3} B_3 & C_7^T A_{7:4} B_4 & C_7^T A_{7:5} B_5 & C_7^T A_{7:6} B_6 & C_7^T A_{7:7} B_7 & \\
  C_8^T A_{8:0} B_0 & C_8^T A_{8:1} B_1 & C_8^T A_{8:2} B_2 & C_8^T A_{8:3} B_3 & C_8^T A_{8:4} B_4 & C_8^T A_{8:5} B_5 & C_8^T A_{8:6} B_6 & C_8^T A_{8:7} B_7 & C_8^T A_{8:8} B_8
\end{array}
\right]$

对于$M$中的任意元素$M_{ij}$，代表的是$x_j$对$y_i$的贡献，通项公式为：$M_{ij}=C_i \times A_{i:j} \times B_j$，其中$A_{i:j}= \prod_{k=j+1}^{i} A_{k}，{j+1<i}$

对于$M$矩阵中的元素，有两种类型，一种是位于对角线上的元素，即$M_{ii}=C_iB_i$，代表$x_t$对当前输出$y_t$的贡献

第二种是位于下三角的元素，即$M_{ij}, i>j$，代表历史$x_j$对当前$y_i$的贡献

采用分块思想，将输入分成多个chunks，假如分成3个chunks，chunk_size为3，那么M矩阵就是一个$3\times3$的分块矩阵，位于对角线区域的块，分别为$M_{00}, M_{11}, M_{22}$，计算公式为

元素形式：$M_{ii}[t, k] = \begin{cases} 
C_t \cdot B_k^\top \cdot \underbrace{e^{L_t - L_k}}_{\text{Decay}} & \text{if } t \ge k \\
0 & \text{if } t < k 
\end{cases}$

矩阵形式：$M_{ii} = \underbrace{\left( C^{(i)} {B^{(i)}}^\top \right)}_{\text{Attention Scores}} \odot \underbrace{\mathbf{D}^{(i)}_{\text{causal}}}_{\text{Mask \& Decay}}$ 其中衰减矩阵是一个下三角矩阵：$\mathbf{D}^{(i)}_{\text{causal}}[t, k] = \begin{cases} e^{L_t - L_k} & t \ge k \\ 0 & t < k \end{cases}$

而位于下三角区域的块，$M_{10}, M_{20}, M_{21}$，计算公式为

元素形式：$M_{ij}[t, k] = \underbrace{\left( C_t \cdot e^{L_t - L_{\text{start}}^{(i)}} \right)}_{\text{Left (Decompression)}} \times \underbrace{\exp\left(L_{\text{start}}^{(i)} - L_{\text{end}}^{(j)}\right)}_{\text{Middle (Passing)}} \times \underbrace{\left( B_k \cdot e^{L_{\text{end}}^{(j)} - L_k} \right)}_{\text{Right (Compression)}}$

矩阵形式：$M_{ij} = \underbrace{\left( C^{(i)} \odot \mathbf{d}_{L}^{(i)} \right)}_{\mathbf{L}_i} \cdot A_{j \to i} \cdot \underbrace{\left( B^{(j)} \odot \mathbf{d}_{R}^{(j)} \right)^\top}_{\mathbf{R}_j^\top}$ 或者 $M_{ij} = \left[ \text{diag}\left(e^{L_{t} - L_{\text{start}}^{(i)}}\right) C^{(i)} \right] \cdot e^{L_{\text{start}}^{(i)} - L_{\text{end}}^{(j)}} \cdot \left[ \text{diag}\left(e^{L_{\text{end}}^{(j)} - L_{k}}\right) B^{(j)} \right]^\top$

## 2. 后向传播

### 1 计算$dz, dSSM, dD, ddAcs$

$O = y \odot z \odot \sigma(z) + D \cdot x$

1. dz

$dz = \delta O \odot y \odot \sigma(z) \left( 1 + z (1 - \sigma(z)) \right)$

2. dD

$dD = \sum_{b, t} \left( \delta O_{b,t} \odot x_{b,t} \right)$

3. dy (用于继续进行后向传播)

$\delta y = \delta O \odot \text{SiLU}(z)$

4. ddA_cs

$\frac{\partial \mathcal{L}}{\partial L_t} = \sum_{p} \left( \delta y_{t,p} \cdot y_{t,p} \right)$

### 2 计算梯度

### 2.1 经过第一步，我们已经得到了SSM的梯度$\partial y$，SSM的计算分为非对角线区域和对角线区域两种情况，先看非对角线区域，其前向传播计算公式为：

$M_{ij}[t, k] = \underbrace{\left( C_t \cdot e^{L_t - L_{\text{start}}^{(i)}} \right)}_{\text{Left (Decompression)}} \times \underbrace{\exp\left(L_{\text{start}}^{(i)} - L_{\text{end}}^{(j)}\right)}_{\text{Middle (Passing)}} \times \underbrace{\left( B_k \cdot e^{L_{\text{end}}^{(j)} - L_k} \right)}_{\text{Right (Compression)}}$

按照反向传播的顺序，最后计算的是左侧部分，那么反向传播就要先计算左侧部分的梯度，左项可以化简为：$$y_{t,p} = \underbrace{\text{Decay}_t}_{\text{标量}} \cdot \sum_{n=1}^{N} (\underbrace{h_{p,n}}_{\text{状态}} \cdot \underbrace{C_{t,n}}_{\text{参数}})$$

那么其梯度公式为：

$$\frac{\partial \mathcal{L}}{\partial h_{p,n}} = \sum_{t=0}^{Q-1} \left( \frac{\partial \mathcal{L}}{\partial y_{t,p}} \cdot \frac{\partial y_{t,p}}{\partial h_{p,n}} \right)$$

因为 $h_{p,n}$ 参与了 $t=0, 1, \dots, Q-1$ 所有时刻的计算，所以每个时刻的 Loss 都要对它负责。

$$dh_{p,n} = \sum_{t=0}^{Q-1} (\text{dout}_{t,p} \cdot \text{Decay}_t) \cdot C_{t,n}$$


关于形状变化：

定义加权后的梯度 $\tilde{D}$：$$\tilde{D}_{t,p} = \text{dout}_{t,p} \cdot \text{Decay}_t$$

形状：[Q, P] (Time, Headdim)现在公式变成了：$$dh_{p,n} = \sum_{t=0}^{Q-1} \tilde{D}_{t,p} \cdot C_{t,n}$$

而C的形状为[Q, N]

消去Q维度，最终的维度正好是$[P, N]$

### 2.2 

假如chunk_size=2, chunk 3的输出为:

$\left\{
\begin{aligned}
y_6 &= C_6 \cdot A_6 \cdot h_5 + C_6 \cdot B_6 \cdot x_6 \\
y_7 &= C_7 \cdot A_7 \cdot A_6 \cdot h_5 + C_7 \cdot B_6 \cdot x_6 + C_7 \cdot B_7 \cdot x_7
\end{aligned}
\right.
$

我们可以得到第k个chunk第i个输出的通用公式：

$ Y_{k}^i = C_{k}^i * dAcs_{k}^i * h_{k-1}^{end} + C_{k}^i * (dAcs_{k}^{i}-dAcs_k^i) * B_k^i * x_k^i$

形状变化为：$[D] = [N] * [1] * [D, N] + [N] * [1] * [N] * [D]$

其中，第一项在N维度上做点积求和，第二项也是在N维度上点积求和。

2.1 所求的梯度是$h_{k-1}^{end}$的梯度，如果有4个chunk，输出为$[y0, y1, y2, y3, y4, y5, y6, y7]，$那么所求的梯度形状为[4, D, N]，是$[dh_{-1}, dh_1, dh_3, dh_5]$

接下来就是递推状态之间的关系：

$h_5 = A_5A_4 * h_3 + A_5* B_4*x_4 + B_5 * x_5$

对于第k个chunk，其状态与第k-1个状态的关系为：

$h_k = dAcs_k^{end} * h_{k-1} + (dAcs_k^{end} - dAcs_k) *B_k * x_k$

形状变化为: $[D, N] = [1] * [D, N] + ([1] - [cs]) * [cs, N] * [cs, D]$

第二项要在cs维度上做点积求和

其中，dAcs的形状为[batch, nheads, nchunks, chunk_size]，比如第4个chunk，dAcs为[A_7, A_7*A_8]

kernel3就是根据kernel2输出的$[dh_{-1}, dh_1, dh_3, dh_5]$，按照倒序的顺序，求$[dh_3, dh_1, dh_-1]$，如果有Dfinal_states, 也就是$dh_7$，那就可以按照公式求解$dh_5$，否则就沿用kernel2的输出结果，如果有Dinitial_states，也就是$dh_{-1}$，那也可以根据公式求$dh_{-3}$。

根据前向公式，可以推导出梯度公式：

$dh_{k-1} = dh_k * dAcs_k^{end} $

$ddAcs_k^{end} = \sum{dh_k * h_{k-1}}$，在D和N维度上求和，但是在kernel3中，可以把D和N维度合并为一个维度。

### 2.3

我们已经把梯度传递到了具体token（或者time）的状态中，接下来就可以把梯度从状态传递到输入x，并且求出 $\Delta$ 和D的梯度

#### 2.3.1 x的梯度dx

dx的梯度来源于两部分，一部分来源于状态方程，一部分来源于对角线区域的输出方程：

假设当前 Chunk 的长度为 $L$，输入为 $X \in \mathbb{R}^{L \times D}$。为了方便矩阵表示，我们要牢记 $\Delta$ 与 $B$ 是解耦的。我们定义中间变量（有效输入）：$$U = \Delta \odot X$$

1. 状态生成路径 (State Equation) —— 影响“下一个 Chunk”当前 Chunk 的输入 $X$，经过 $B$ 投影和 $A$ 的衰减，最终累积成了一个状态 $h_{end}$，传给下一个 Chunk。标量形式：$$h_{end} = \sum_{t=0}^{L-1} \underbrace{\exp(\text{SegSum}(A)_{t \to end})}_{\text{Decay}} \cdot B_t \cdot (\Delta_t x_t)$$矩阵形式：$$h_{end} = (\mathbf{decay}_{last} \odot B)^T \cdot U$$

2. 块内输出路径 (Intra-chunk Equation) —— 影响“当前 Chunk 输出”当前 Chunk 的输入 $X$，在块内进行因果卷积（下三角扫描），生成当前块的输出 $Y$。矩阵形式：$$Y = \left( \underbrace{\mathbf{M}_{causal} \circ \mathbf{M}_{decay} \circ (CB)}_{\text{Lower Triangular Masked Matrix}} \right) \cdot U + D \cdot X$$

$\mathbf{M}_{causal}$: 下三角掩码（$t \ge k$）。

$CB$: $C$ 与 $B$ 的矩阵乘积张量。


知道前向传播方程之后，求梯度就很简单了，分别求就行：

求 $dx$ 的过程，就是把上述两条路径的梯度“收集”回来的过程。根据链式法则，我们先求对中间变量 $U$ 的梯度，再求对 $X$ 的梯度。定义 Accumulator (Acc) 为 Loss 对中间变量 $U$ 的梯度：$$\text{Acc} = \frac{\partial \mathcal{L}}{\partial U}$$

1. 来自状态的梯度 (From Next Chunk)对应代码 Phase 2。Loss 对 $h_{end}$ 的梯度是 dstates。梯度沿着状态生成路径倒流：$$\text{Acc}_{state} = \underbrace{B}_{\text{Proj}} \cdot \underbrace{(\mathbf{decay}_{last} \odot dstates)}_{\text{Decayed Gradient}}$$(注：代码中体现为 tl.dot(b, dstates * scale))

2. 来自输出的梯度 (From Intra-chunk Output)对应代码 Phase 3。Loss 对 $Y$ 的梯度是 dout。梯度沿着块内矩阵乘法倒流。关键点：前向是 $Y = M \cdot U$，后向则是 $\text{Acc} = M^T \cdot dout$。下三角矩阵 $M$ 的转置 $M^T$ 变成了上三角矩阵！$$\text{Acc}_{scan} = \underbrace{\left( \mathbf{M}_{causal}^T \circ \mathbf{M}_{decay}^T \circ (CB)^T \right)}_{\text{Upper Triangular (Reverse Scan)}} \cdot dout$$(注：代码中通过 stride 技巧隐式转置了 $CB$，并通过 k >= m 实现了上三角掩码)

3. 最终合并：求 $dx$对应代码 Phase 4。现在我们有了总的 $\text{Acc} = \text{Acc}_{state} + \text{Acc}_{scan}$。根据 $U = \Delta \odot X$ 和直连通路 $D \cdot X$：$$dx = \frac{\partial \mathcal{L}}{\partial U} \cdot \frac{\partial U}{\partial X} + \frac{\partial \mathcal{L}}{\partial Y_{direct}} \cdot \frac{\partial Y_{direct}}{\partial X}$$最终矩阵公式：$$\mathbf{dx} = \underbrace{\text{Acc} \odot \Delta}_{\text{SSM Path}} + \underbrace{dout \odot D}_{\text{Skip Path}}$$

#### 2.3.2 D和$\Delta$的梯度

我们知道

$out = SSM(x) + D*x$

所以D的梯度为：

$dD = \sum(dout * x)$

这里，D的维度为[nheads, head_dim]或者[nheads,]。dout的形状为[batch, seq_len, nheads, head_dim], x的形状为[batch, seq_len, nheads, head_dim]

我们定义triton的grid的三个轴分别为triton.cdiv(chunk_size, META['BLOCK_SIZE_M']) * triton.cdiv(head_dim, META['BLOCK_SIZE_N']), batch * nchunks, nheads，

不考虑batch，nchunks和nheads维度，对于chunk_size和head_dim中的特定Block:(pid_m, pid_n)，我们需要对dout * x的结果进行归约，如果D的维度为[nheads, head_dim]，那么就在axis=0上归约，如果为[nheads,]就全部归约。

根据公式：

$u = \Delta * x$

可以知道:

$d\Delta = \sum(du * x)$

其中，du就是前面求出来的acc，$d\Delta$的形状为[batch, nchunks, nheads, chunk_size]，所以要在head_dim维度上归约。对于相同chunk_size上的Block，不同head_dim上的Block，计算出来的$d\Delta$需要存入相同的指针地址，为了避免竞争，使用tl.atomic_add进行存储。

总结一下在写这个kernel时遇到的一些高性能技巧


1. 规约策略：空间换时间 (Scatter-Reduce / Global Accumulation)
场景：需要计算全局梯度（如 dD），涉及 Batch、Chunk、Time 所有维度的求和。 痛点：如果成千上万个 Thread Block 同时使用 atomic_add 往同一个地址（例如 (nheads, hdim)）写入，会导致严重的原子锁竞争 (Atomic Contention)，流水线停顿，性能极差。

技巧 (The dD Trick)：

分两步走：

Kernel 内（Scatter）：给每个计算单元（Grid 节点）分配一块“私有显存”。

定义一个高维 Tensor（例如 5D），包含 [Num_Time_Blocks, Batch, Chunk, ...]。

每个 Block 只写自己的位置，完全无锁（Lock-free），使用极快的 store。

Kernel 外（Reduce）：利用 PyTorch 的高度优化算子进行求和。

dD.sum(dim=(0, 1, 2))。

核心思想：通过增加显存占用（空间），消除原子锁竞争（时间），大幅提升吞吐量。


2. 并发安全：原子加 vs. 直接存 (Atomic Add vs. Store)
场景：多个 Block 需要写入同一个输出 Tensor。 法则：看 Grid 切分了哪个维度，以及我们在哪个维度上求和。

比如在求ddt时，使用了

ddt = tl.sum(acc * x, axis=1)

acc和x都是在chunk_size和head_dim维度上进行切分的，ddt在head_dim维度上进行了求和，而ddt的存储地址为

ddt_ptrs = ddt_ptr + offs_m * stride_ddt_csize

也就是说，对于相同的offs_m偏移时，不同的offs_n偏移的kernel会同时往ddt_ptrs地址中存数据，此时就需要要原子加，在其中一个kernel访问该地址时，其他kernel要等候。

而对于常规kernel的数据存储地址，每个kernel的存储地址不同，所以不存在竞争的问题，此时用tl.store

3. 访存魔法：零开销转置 (Stride Trick / Zero-overhead Transpose)场景：计算矩阵乘法 $Acc += CB^T \cdot dout$。物理内存中存的是 $CB$，但我们需要 $CB^T$。痛点：在 Kernel 里手动交换索引（val[k, m]）写起来麻烦，且显式转置数据非常慢。技巧：修改元数据，不动数据：在 Python 端调用 Kernel 时，交换传入的 Stride 参数。传给 Kernel 的 stride_row其实是物理的 stride_col。传给 Kernel 的 stride_col其实是物理的 stride_row。效果：Kernel 代码依然写 ptr + m*stride_m + k*stride_k，但硬件读出来的数据已经是转置好的。核心思想：利用 View（视图）的灵活性，避免 Data Movement（数据搬运）。

4. 计算优化：指针跳跃 (Pointer Jumping for Causal Mask)场景：处理因果掩码（Causal Mask, $k \ge m$）或三角矩阵。痛点：朴素做法是从 $k=0$ 循环到 $K$，然后在循环体内用 if k < m: mask=0。这导致大量的无效内存读取和无效计算（乘 0）。技巧：直接跳过无效区：利用 Grid 信息知道当前 Block 负责的起始时间 $m_{start}$ (K_MIN)。初始化指针时，直接 ptr += K_MIN * stride。循环直接从 K_MIN 开始。核心思想：只计算有效数据。对于下三角/上三角计算，这能节省接近 50% 的算力。

5. 数值稳定性：对数空间的安全操作场景：计算指数衰减 $\exp(\text{cumsum}_k - \text{cumsum}_m)$。技巧：钳位 (Clamping)：tl.exp(tl.minimum(val, 0.0))。原因：虽然理论上 $k \ge m$ 时差值应 $\le 0$，但浮点误差可能导致微小的正数，导致 exp 结果 $>1$（梯度爆炸）。强制截断到 0 保证数值安全。

6. 维度切分：Tiling (分块)场景：特征维度 headdim 或状态维度 dstate 很大，单个 Block 的寄存器/Shared Memory 放不下。技巧：Grid 切分：不仅切分 Batch 和 Time，还切分 Feature (pid_n)。循环切分：在 Kernel 内部，对 dstate 维度 (K) 进行循环迭代 (for k in range(0, dstate, BLOCK_K))。核心思想：分而治之。无论 Tensor 多大，都把它切成适合 GPU 硬件单元（Tensor Core / SRAM）大小的小块来处理。

### 2.4 B的梯度

同样是根据状态方程：

$$h_{end} = (\mathbf{decay}_{t->last} \odot B)^T \cdot U$$




求得B的梯度：

$$dB = dh_{end} * decay_{t->end} * U$$

dB的形状为[batch, seq_len, ngroups, d_state]，dstates为[batch, nchunks, nheads, head_dim, d_state], U为[batch, seq_len, nheads, head_dim]

grid的划分为：triton.cdiv(chunk_size, META['BLOCK_SIZE_M']) * triton.cdiv(dstate, META['BLOCK_SIZE_N']), batch * nchunks, nsplits * ngroups

不考虑batch和nchunks以及nheads，U: [cs, head_dim], dstates: [head_dim, d_state]

所以，grid的第一个轴是根据chunk_size和d_state进行划分的。

除此之外，这里需要注意GQA机制，也就是说ngroups对应nheads，一个group对应nheads // ngroups个头。我们把同一个group下的nheads划分为多个Blocks，也就是split个block，一共有ngroups * split个block，同一个block中，把该block中的nheads的dB相加。
